In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import math
import pdb
from matplotlib import pyplot as plt
import importlib
import seaborn as sns

from cartopy import crs as ccrs
from cartopy import feature as cfeature
from matplotlib import cm

import loaders
import geography
import visualization

importlib.reload(loaders)
importlib.reload(geography)
importlib.reload(visualization)
import visualization as vis

%matplotlib

Using matplotlib backend: Qt5Agg


In [4]:
fn = './tables/TableS3.csv'
df = pd.read_csv(fn)
df

,Unnamed: 0,Origin,Dest,Month,Prediction,lower,upper,origin_lon,origin_lat,dest_lon,dest_lat,dest_p_no_outbreak,risk_ij,risk_i,dest_municipality,dest_name,dest_region,dest_continent
0,0,CTS,HND,10,527281,483306,575257.0,141.68,42.79,139.78,35.55,0.690671,0.309329,0.367017,Tokyo,Tokyo Haneda International Airport,JP-13,AS
1,1,DEL,BOM,10,303139,276824,331955.0,77.10,28.56,72.87,19.10,0.773800,0.226200,0.475864,Mumbai,Chhatrapati Shivaji International Airport,IN-MM,AS
2,2,PEK,SHA,10,298365,274382,325928.0,116.59,40.08,121.34,31.20,0.779720,0.220280,0.744630,Shanghai,Shanghai Hongqiao International Airport,CN-31,AS
3,3,FUK,HND,10,324308,297075,354037.0,130.44,33.58,139.78,35.55,0.796421,0.203579,0.285490,Tokyo,Tokyo Haneda International Airport,JP-13,AS
4,4,SHA,PEK,10,279779,257300,304221.0,121.34,31.20,116.59,40.08,0.801104,0.198896,0.383221,Beijing,Beijing Capital International Airport,CN-11,AS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20948,20948,OKC,DTW,10,4017,3860,4181.0,-97.60,35.40,-83.36,42.21,1.000000,0.000000,0.002952,Detroit,Detroit Metropolitan Wayne County Airport,US-MI,NaN
20949,20949,OMA,DTW,10,5546,5302,5801.0,-95.90,41.30,-83.36,42.21,1.000000,0.000000,0.008035,Detroit,Detroit Metropolitan Wayne County Airport,US-MI,NaN
20950,20950,ORD,DTW,10,42045,40566,43578.0,-87.99,41.98,-83.36,42.21,1.000000,0.000000,0.240523,Detroit,Detroit Metropolitan Wayne County Airport,US-MI,NaN
20951,20951,ORF,DTW,10,6077,5834,6331.0,-76.21,36.90,-83.36,42.21,1.000000,0.000000,0.015498,Detroit,Detroit Metropolitan Wayne County Airport,US-MI,NaN


In [17]:
plt.close('all')
def plot_geodesics(df, region, opacity_s=0.8):
    print("Plotting geodesics")
    #df = df.query('Dest in @destinations')
    fig = plt.figure(figsize=vis.MAP_SIZE)
    plateCr = ccrs.PlateCarree()
    plateCr._threshold = plateCr._threshold/10.
    ax = plt.axes(projection=plateCr)
    
    #mx = df.Prediction.max()
    if region == 'global':
        cutoff = 0.005
    else:
        cutoff = 0.1
    opacity = np.maximum(df.risk_ij, cutoff)
    lines = plt.plot(df[['origin_lon', 'dest_lon']].T,
                     df[['origin_lat', 'dest_lat']].T, 
                     color='r',
                     transform=ccrs.Geodetic())
    print('scalin')
    [line.set_alpha(vis.opacity_scaler(alpha, opacity_s)) for alpha, line in zip(opacity, lines)]
    
    vis._annotate(ax, text='a', color='k', region=region)
    vis._add_features(ax, region=region)

    plt.tight_layout()
plot_geodesics(df, region, opacity_s=1.5)

Plotting geodesics
scalin


(1001, 2001)

In [45]:
vert_window = 7
horz_window = 11
arr = np.ones((vert_window*2+1, horz_window*2+1))
X, Y = np.meshgrid(np.arange(-horz_window,horz_window+1), np.arange(-vert_window, vert_window+1))
inds = Y**2/vert_window**2 + X**2/horz_window**2 > 1
arr[inds] = 0 
plt.imshow(arr)
plt.show()
arr.shape

(15, 23)

(1001, 2001)

masked_array(data=[0.52656971, 0.66219197, 0.98766947, ..., 0.15271373,
                   0.01485872, 0.03986964],
             mask=False,
       fill_value=1e+20)

In [43]:
plt.close('all')
x = np.linspace(0,1)
f = lambda x,s=1: np.maximum(np.arctan(1/(1-x)**s - 1/x**s) / np.pi + 0.5, 0.05)
y = f(x)
plt.plot(x,y)
p = 0.75
plt.scatter(p, f(p),color='r')
plt.show()
f(p)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


0.8857997487800919

In [31]:
y

array([0.        , 0.00663338, 0.01356146, 0.02081828, 0.0284436 ,
       0.03648417, 0.04499516, 0.0540421 , 0.06370317, 0.07407217,
       0.0852622 , 0.09741033, 0.11068332, 0.12528468, 0.14146303,
       0.15952135, 0.17982617, 0.20281377, 0.22898749, 0.25889544,
       0.29307025, 0.33190868, 0.3754773 , 0.42327455, 0.47406225,
       0.52593775, 0.57672545, 0.6245227 , 0.66809132, 0.70692975,
       0.74110456, 0.77101251, 0.79718623, 0.82017383, 0.84047865,
       0.85853697, 0.87471532, 0.88931668, 0.90258967, 0.9147378 ,
       0.92592783, 0.93629683, 0.9459579 , 0.95500484, 0.96351583,
       0.9715564 , 0.97918172, 0.98643854, 0.99336662, 1.        ])

In [ ]:
x**2 + (y-1)**2 = 1
y-1 = -(1-x**2) + 1